In [ ]:
token ='insert_your_token_here'

from neuprint import Client

c = Client('neuprint.janelia.org', dataset='manc:v1.2.1', token=token)
c.fetch_version()
from neuprint import fetch_adjacencies
from neuprint import merge_neuron_properties
from neuprint import fetch_neurons, NeuronCriteria as NC, fetch_synapses, SynapseCriteria as SC, skeleton_segments, fetch_synapse_connections

from neuprint.utils import connection_table_to_matrix

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

from sklearn.metrics.pairwise import cosine_similarity
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering


In [ ]:
md_4_neurons = [15263,
 18951,
 21105,
 21209,
 21219,
 21773,
 21976,
 22045,
 22090,
 22123,
 22217,
 22228,
 22247,
 22411,
 22543,
 22613,
 22719,
 22995,
 23476,
 23532,
 23783,
 24165,
 24180,
 25494,
 25636,
 26664,
 101527,
 152738,
 154741,
 163802]


criteria = NC(bodyId=md_4_neurons)
sn_df, sn_conn_df = fetch_adjacencies(criteria)

neuron_info, _ = fetch_neurons(criteria)
left_sn = neuron_info[neuron_info.instance.str.contains('_L')].bodyId.unique()
#right_sn = sn_df[sn_df.instance_pre.str.contains('R')].bodyId_pre.unique()


In [ ]:
order = sn_df[sn_df.bodyId.isin(md_4_neurons)][sn_df.instance.fillna('').str.contains('_L')].sort_values('instance', ascending=True).bodyId.unique()

dv = [163802,  23476,  23783,  24180,  26664,  21976,  22123,
       23532,  18951,  25636,  21773,  22045,  22090, 154741,  22247]
relation = sn_conn_df[sn_conn_df.bodyId_post.isin(left_sn)].pivot_table(index='bodyId_pre', columns='bodyId_post', 
                                               values='weight', aggfunc='sum').fillna(0)
relation = relation[dv].reindex(dv)

r = relation.drop_duplicates().copy()


fig = plt.figure(figsize=(6,6))
#plt.rcParams["font.size"] = 5
sns.heatmap(r, vmin=0,vmax=20, square=True, cmap='Greys')
plt.show()

In [ ]:
order = sn_df[sn_df.bodyId.isin(md_4_neurons)][sn_df.instance.fillna('').str.contains('_L')].sort_values('instance', ascending=True).bodyId.unique()

ap = [18951, 23532, 25636, 163802, 
      21773, 22045, 22090, 23476, 23783, 24180, 26664, 154741,
      21976, 22123, 22247]
relation = sn_conn_df[sn_conn_df.bodyId_post.isin(left_sn)].pivot_table(index='bodyId_pre', columns='bodyId_post', 
                                               values='weight', aggfunc='sum').fillna(0)
relation = relation[ap].reindex(ap)

r = relation.drop_duplicates().copy()


fig = plt.figure(figsize=(6,6))
#plt.rcParams["font.size"] = 5
sns.heatmap(r, vmin=0,vmax=20, square=True, cmap='Greys')
plt.show()

In [ ]:
synapses = fetch_synapse_connections(NC(bodyId=left_sn))

synapses['pre_point'] = synapses.apply(lambda row: [row['x_pre'], row['y_pre'], row['z_pre']], axis=1)
synapses['post_point'] = synapses.apply(lambda row: [row['x_post'], row['y_post'], row['z_post']], axis=1)

In [ ]:
def filt_syn_df(syn_df, syn_thresh):
    
    indices_to_include = []

    # Find unique presynaptic neurons
    unique_pre_ids = syn_df.bodyId_pre.unique().tolist()

    # Loop through presynaptic neurons
    for i in unique_pre_ids:       
        pre_df = syn_df[syn_df.bodyId_pre == i]
        # Find unique postsynaptic neurons targeted by i-th presynaptic neuron
        unique_post_ids = pre_df.bodyId_post.unique().tolist() 
        # Loop through postsynaptic neurons
        for j in unique_post_ids:    
            # Is number of synapses onto j-th postsynaptic neuron larger than or equal to syn_thresh? 
            if sum(pre_df.bodyId_post == j) >= syn_thresh: 
                # Get indices (rows)
                indices = pre_df.index.values[pre_df.bodyId_post == j]       
                # Loop through indices 
                for k in indices:
                    # Append each index separately to avoid lists within list
                    indices_to_include.append(k)

    # Sort indices in ascending order
    indices_to_include.sort()
    
    # Create new syn_df with only connections above syn_thresh
    syn_df_filt = syn_df.loc[indices_to_include]
    
    return syn_df_filt

#functions adapted from FANC

import networkx as nx

def get_networkx_graph(syn_df, source = 'bodyId_pre', 
                       target = 'bodyId_post', edge_attr = 'count'):
    
    
    edge_df = syn_df.groupby([source, target]).size().sort_values(ascending=False).reset_index(name=edge_attr)
        
    graph = nx.from_pandas_edgelist(edge_df, source=source, 
                                  target=target, edge_attr=edge_attr)
    return graph

def get_edge_df(syn_df, source = 'bodyId_pre', 
                target = 'bodyId_post', edge_attr = 'count'):

    
    edge_df = syn_df.groupby([source, target]).size().sort_values(ascending=False).reset_index(name=edge_attr)
    
    
    return edge_df

def get_asymm_conn_mat(syn_df,  weight='count'):
    
    g = get_networkx_graph(syn_df,  edge_attr=weight)
    
    edge_df = get_edge_df(syn_df,  edge_attr=weight)
    
    targets = list(edge_df.bodyId_post.unique())
    sources = list(edge_df.bodyId_pre.unique())
    
    # Get indices for g.nodes indicating which node IDs are sources and which are targets
    source_ix, target_ix = [], []
    source_ids, target_ids = [], []
    for ix, i in enumerate(g.nodes()):
        if i in sources:
            source_ix.append(ix) 
            source_ids.append(i) 
        if i in targets:
            target_ix.append(ix)
            target_ids.append(i)

    conn_mat = nx.to_numpy_array(g, weight=weight) # Full connectivity matrix
#     conn_mat = nx.DiGraph(g, weight=weight)
    asymm_conn_mat = conn_mat[source_ix,:]
    asymm_conn_mat = asymm_conn_mat[:,target_ix]
    
    return asymm_conn_mat, source_ids, target_ids

def get_clustered_order(sim_mat, distance_threshold = 0, 
                       n_clusters = None, **kwargs):
    
    model = AgglomerativeClustering(distance_threshold=distance_threshold, n_clusters=None).fit(sim_mat)
    #create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  #leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    #plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)
    dend_dict = dendrogram(linkage_matrix, **kwargs)
    
    #sorted order of indices found through clustering
    clustered_order = dend_dict['ivl']
    
    parsed_order = []
    for c in clustered_order:
        if '(' in c:
            c = c.split('(')[1]
            c = c.split(')')[0]
        parsed_order.append(int(c))
    
    return parsed_order

def get_sim_matrix(syn_df, asymmetric = True, weight='count'):
    
    #get weighted connectivity matrix from networkx graph
    if asymmetric:
        conn_mat, sources, targets = get_asymm_conn_mat(syn_df, weight=weight)
    else:
        g = get_networkx_graph(syn_df, edge_attr=weight)
        conn_mat = nx.to_numpy_array(g, weight=weight)
     
    conn_mat = np.asarray(conn_mat) #np.matrix usage is deprecated; convert to np.array

    #calculate cosine similarity
    sim_mat = cosine_similarity(conn_mat)
    
    return conn_mat, sim_mat

def reorder_by_cosine(syn_df, weight ='count', 
                      distance_threshold = 0, 
                      n_clusters = None,
                      asymmetric = True,
                      column_order = 'cosine'):

    #get weighted connectivity matrix from networkx graph
    if asymmetric:
        conn_mat, sources, targets = get_asymm_conn_mat(syn_df, weight=weight) 
    else:
        g = get_networkx_graph(syn_df, edge_attr=weight)
        conn_mat = nx.to_numpy_array(g, weight=weight)
        
    conn_mat = np.asarray(conn_mat) #np.matrix usage is deprecated; convert to np.array
    
    #calculate cosine similarity
    sim_mat = cosine_similarity(conn_mat)
    
    #clustering based on cosine similarity
    row_ordered = get_clustered_order(sim_mat, distance_threshold=distance_threshold, n_clusters=n_clusters) 
    
    if asymmetric==True and column_order == 'cosine':
        column_sim_mat = cosine_similarity(conn_mat.T)
        column_ordered = get_clustered_order(column_sim_mat, distance_threshold=distance_threshold, n_clusters=n_clusters)
    elif asymmetric == False and column_order == 'cosine':
        column_ordered = row_ordered
    else:
        column_ordered = column_order
    
    #reordering connectivity matrix 
    conn_mat = conn_mat[row_ordered, :]
    conn_mat = conn_mat[:, column_ordered]

    #reordering source similarity matrix
    sim_mat = sim_mat[row_ordered, :]
    sim_mat = sim_mat[:, row_ordered] 
        
    #reordering source and target IDs
    sources_ordered = [sources[i] for i in row_ordered] 
    targets_ordered = [targets[i] for i in column_ordered] 
     
    return conn_mat, sim_mat, sources_ordered, targets_ordered


def reorder_by_custom_order(syn_df, 
                            custom_row_order=None, 
                            custom_col_order=None, 
                            weight='count', 
                            asymmetric=True):
    """
    Reorders the connectivity and similarity matrices based on custom-defined row and column orders.
    
    Parameters:
    - syn_df: DataFrame with synapse information.
    - custom_row_order: List of source neuron IDs in desired row order.
    - custom_col_order: List of target neuron IDs in desired column order.
    - weight: Edge attribute used for weights (default 'count').
    - asymmetric: Whether to treat the matrix as asymmetric (default True).
    
    Returns:
    - conn_mat_reordered: Reordered connectivity matrix.
    - sim_mat_reordered: Reordered cosine similarity matrix.
    - sources_ordered: Ordered list of source neuron IDs.
    - targets_ordered: Ordered list of target neuron IDs.
    """

    # Get connectivity matrix and source/target IDs
    if asymmetric:
        conn_mat, sources, targets = get_asymm_conn_mat(syn_df, weight=weight)
    else:
        g = get_networkx_graph(syn_df, edge_attr=weight)
        conn_mat = nx.to_numpy_array(g, weight=weight)
        sources = targets = list(g.nodes())

    conn_mat = np.asarray(conn_mat)
    sim_mat = cosine_similarity(conn_mat)

    # Build index maps for custom ordering
    if custom_row_order is not None:
        row_indices = [sources.index(i) for i in custom_row_order]
    else:
        row_indices = list(range(len(sources)))

    if custom_col_order is not None:
        col_indices = [targets.index(j) for j in custom_col_order]
    else:
        col_indices = list(range(len(targets)))

    # Reorder matrices
    conn_mat_reordered = conn_mat[row_indices, :][:, col_indices]
    sim_mat_reordered = sim_mat[row_indices, :][:, row_indices]

    # Get the ordered sources/targets
    sources_ordered = [sources[i] for i in row_indices]
    targets_ordered = [targets[j] for j in col_indices]

    return conn_mat_reordered, sim_mat_reordered, sources_ordered, targets_ordered

In [ ]:
syn_thresh = 4
synapse_df = filt_syn_df(synapses, syn_thresh)

filtered_synapse_df = synapse_df[synapse_df['bodyId_post'].isin(sn_conn_df[sn_conn_df.weight>=syn_thresh].bodyId_post.unique())]
#plt.figure(figsize=(20,20))

conn_mat, sim_mat, pre_ids_ordered, post_ids_ordered = reorder_by_cosine(filtered_synapse_df)

In [ ]:

conn_mat, sim_mat, src_ordered, tgt_ordered = reorder_by_custom_order(
    filtered_synapse_df,
    custom_row_order=dv,
    custom_col_order=dv
)

In [ ]:
sns.set_context('talk')
fig = plt.figure(figsize=(5,5))
sns.heatmap(sim_mat, vmin=0.5, square=True, cmap='Greys', cbar=False)
plt.xlabel('md')
plt.ylabel('md')
plt.xticks(rotation = 90)
plt.rcParams.update({'font.size': 11})
plt.show()